# Problem Statement

## Objective: 
- Minimize the total operational costs, which may include fuel costs, crew costs, and delay costs, or maximize revenue.
## Constraints:
- **Aircraft availability:** Each aircraft can only be used for one flight at a time.
- **Flight schedule**
- **Crew assignment:** Each aircraft must have at least one crew


# Mathematical Formulation

**Index**:
- $i \in F $: Index of flights
- $a $: Index of aircraft
- $c$: Index of crew


**Decision Variables**:
- $ x_{ia} $: Binary decision variable indicating flight $i$ is assigned to aircraft $a$
- $ y_{ic} $: Binary decision variable indicating crew $c$ is assigned to flight $i$


**Objective Function**:
$$ \text{Minimize } \sum_{i} \sum_{a} CA_{ia} x_{ia} + \sum_{i} \sum_{c} CC_{ic} y_{ic} $$

**Constraints**:
1. **Aircraft availability**:
$$ \sum_{a} x_{ia} = 1  \quad  \forall i$$

2. **Flight constraint**:
$$ \sum_{i} x_{ia} \leq 1 \quad \forall a $$

3. **Crew assignment**:
$$ \sum_{c} y_{ic} = 1 \quad \forall i  $$

4. **Range Constraint**
$$ x_{ia}, y_{ic} = \{0,1\}  $$

# Pyomo implementation

In [3]:
# !pip install pyomo

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Define the sets
flights = range(1, 11)  # Example: 10 flights
aircraft = range(1, 6)  # Example: 5 aircraft
crew = range(1, 8)      # Example: 7 crew members

# Define the parameters
c = {(i, a): 100 + i * a for i in flights for a in aircraft}  # Cost of assigning flight to aircraft
d = {(i, c): 50 + i * c for i in flights for c in crew}       # Cost of assigning flight to crew

# Create a model
model = ConcreteModel()

# Define decision variables
model.x = Var(flights, aircraft, within=Binary)
model.y = Var(flights, crew, within=Binary)

# Objective function
model.obj = Objective(expr=sum(c[i, a] * model.x[i, a] for i in flights for a in aircraft) +
                           sum(d[i, c] * model.y[i, c] for i in flights for c in crew), 
                      sense=minimize)

# Constraints
# Each flight is assigned to exactly one aircraft
model.flight_assignment = ConstraintList()
for i in flights:
    model.flight_assignment.add(sum(model.x[i, a] for a in aircraft) == 1)

# Each aircraft can be assigned to at most one flight at any time
# (for simplicity, ignoring time slots in this example)
model.aircraft_usage = ConstraintList()
for a in aircraft:
    model.aircraft_usage.add(sum(model.x[i, a] for i in flights) <= 1)

# Crew assignment (each flight needs at least one crew)
model.crew_assignment = ConstraintList()
for i in flights:
    model.crew_assignment.add(sum(model.y[i, c] for c in crew) >= 1)

# Solve the model using CPLEX
opt = SolverFactory('cplex')
results = opt.solve(model, tee=True)

# Display results
model.pprint()

# Extract the solution
assigned_flights = [(i, a) for i in flights for a in aircraft if model.x[i, a].value == 1]
assigned_crew = [(i, c) for i in flights for c in crew if model.y[i, c].value == 1]

print("Assigned Flights to Aircraft:")
for i, a in assigned_flights:
    print(f"Flight {i} -> Aircraft {a}")

print("Assigned Crew to Flights:")
for i, c in assigned_crew:
    print(f"Flight {i} -> Crew {c}")
